In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.io import wavfile
from echohiding import echo_hide, extract_echo_bits, get_odg_distortion, get_mp3_encoded

In [ ]:
# Load audio samples from a randomly chosen example from OpenMic Dataset
sr, x = wavfile.read("086799_0.wav")
x = np.mean(x, axis=1) # Mix to mono
x = x/32768 # Put in range [-1, 1]
ipd.Audio(x, rate=sr)

In [3]:
# Make some random bits
np.random.seed(0)
b = np.random.randint(0, 2, 430)

# Choose window length for echoes.  Higher window lengths
# may lead to more robust encoding, but at a lower bitrate
L = 1024
print("Payload: {} bits per second".format(sr//L))

Payload: 43 bits per second


In [ ]:
# Hide echoes. A higher alpha will make the echoes more obvious
y = echo_hide(x, L, b, alpha=0.2)
ipd.Audio(y, rate=sr)

In [5]:
# Robustness
b_est = extract_echo_bits(y, L)
berr = 1-np.sum(b==b_est)/b.size
print("Bit error rate wav encode:", berr)

# Robustness
bitrate = 64
b_est = extract_echo_bits(get_mp3_encoded(y, sr, bitrate), L)
berr = 1-np.sum(b==b_est)/b.size
print("Bit error rate {}kbps mp3:".format(bitrate), berr)

Bit error rate wav encode: 0.06511627906976747
Bit error rate 64kbps mp3: 0.15116279069767447


In [6]:
# Imperceptibility (using Holters GstPEAQ)
odg, di = get_odg_distortion(x, y, sr)
print("Objective difference grade: {:.3f}".format(odg))
print("Distortion index: {:.3f}".format(di))

Objective difference grade: -0.385
Distortion index: 1.781
